![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Routing Optimization

Implementing a semantic router is a great light weight way to add branching logic to your application without taking on additional LLM calls. However, it can be tough to determine the optimal distance threshold values for your routes to maximize performance. This guide will walk through:

- how to configure a semantic router
- how to optimize the distance thresholds for the routes
- a comparison between performing similar logic with an LLM versus a router

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/semantic-router/01_routing_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Setup

## Install Packages

In [ ]:
%pip install -q sentence-transformers ranx sentence-transformers "redisvl>=0.6.0" "redis-retrieval-optimizer>=0.4.2"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Grab data (if colab)

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/semantic-router/resources .
!rm -rf temp_repo

## Run a Redis instance

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [3]:
import os
import warnings

warnings.filterwarnings("ignore")

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Routing with multiple routes

## Define the Routes

Below we define 3 different routes. One for `faq` (frequently asked questions), one for `general`, and
another for `blocked`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route. Note that each route can have it's own distinct `distance_threshold` that defines what is considered a match for the particular query. 

In [4]:
from redisvl.extensions.router import Route

faq = Route(
    name="faq",
    references=[
        "How do I reset my password?",
        "Where can I view my order history?",
        "How do I update my shipping address?",
        "Where are my saved payment methods?",
        "How do I change my email preferences?",
        "How can I see my loyalty points balance?",
        "Where do I find my digital receipts?",
        "How do I enable two-factor authentication?",
        "Can I change my username or email?",
        "How do I manage my account settings?"
    ],
    metadata={"category": "account_management", "priority": 1},
    distance_threshold=0.5
)

general = Route(
    name="general",
    references=[
        "I received the wrong item in my order, can you help?",
        "Can you recommend products that match my specific needs?",
        "The assembly instructions for my furniture are unclear",
        "I need help finding a product with particular specifications",
        "My order arrived damaged, what are my options?",
        "Can you help me design a room with your products?",
        "I'm looking for custom sizing options for this product",
        "The item I received doesn't match the online description",
        "I need advice on which model would work best for my situation",
        "Can you help troubleshoot an issue with my recent purchase?"
    ],
    metadata={"category": "customer_service", "priority": 2},
    distance_threshold=0.5
)

blocked = Route(
    name="blocked",
    references=[
        "What is your company's stance on the recent election?",
        "Do you support liberal or conservative policies?",
        "Can you tell me another customer's address?",
        "What's your CEO's opinion on gun control?",
        "I need personal information about one of your employees",
        "How does your company vote on political issues?",
        "Can you provide me with someone's credit card details?",
        "What's your position on immigration reform?",
        "I want to know where a specific customer lives",
        "Does your company donate to political campaigns?"
    ],
    metadata={"category": "prohibited", "priority": 3},
    distance_threshold=0.5
)

## Initialize the SemanticRouter

Like before the ``SemanticRouter`` class will automatically create an index within Redis upon initialization for the route references.

In [5]:
import os
from redisvl.extensions.router import SemanticRouter

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
ecom_router = SemanticRouter(
    name="ecom-router",
    routes=[faq, general, blocked],
    redis_url="redis://localhost:6379",
    overwrite=True # Blow away any other routing index with this name
)

13:22:06 datasets INFO   PyTorch version 2.3.0 available.
13:22:06 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
13:22:06 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

13:22:08 redisvl.index.index INFO   Index already exists, overwriting.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## View the created index

In [6]:
# look at the index specification created for the semantic router
!rvl index info -i ecom-router



Index Information:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────╮
│ Index Name      │ Storage Type    │ Prefixes        │ Index Options   │ Indexing        │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┤
| ecom-router     | HASH            | ['ecom-router'] | []              | 0               |
╰─────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────╯
Index Fields:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────╮
│ Name            │ Attribute       │ Type            │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼──────────────

## Test it out

The router will automatically vectorize the query perform a search to find the most related topic and return the related route match.

In [7]:
# Query the router with a statement
route_match = ecom_router("Whatup how do i reset my password?")
route_match

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

RouteMatch(name='faq', distance=0.108501493931)

## Optimize route distance thresholds with test data

For optimization within redisvl you can create test data manually or make use of a model to generate some for you. In this case we will use a model to do it for us.

Prompt for creating test data:
> used claude sonnet 3.7 for generation of resource

```txt
You are a test data creation helper. 

Create test data of the form:

{
    "query": "query about a topic",
    "query_match": "topic-the-query-matches"
}

The 3 available topics are: faq, general, and blocked. Generate many examples that map to these topics such that we can train a model to find the best thresholds for this classification task. Also make sure to include some examples that don't map to any of the topics to check the null case for these leave the query_match field empty.
```

The output of this call was saved to `./resources/test_data.json`

In [8]:
import json

with open("resources/ecom_train_data.json", "r") as f:
    train_data = json.load(f)

## Run optimization with router

Using the `RouterThresholdOptimizer` from the `redis-retrieval-optimizer` library.

In [9]:
from redis_retrieval_optimizer.threshold_optimization import RouterThresholdOptimizer

optimizer = RouterThresholdOptimizer(ecom_router, train_data)
optimizer.optimize()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Eval metric F1: start 0.867, end 0.956 
Ending thresholds: {'faq': 0.6838383838383841, 'general': 0.47171717171717187, 'blocked': 0.6151515151515151}


## Test classification against LLM

Using the same prompt above we generated and stored another 20 questions to use as our `test_data` to compare against using an LLM model to perform this classification.

In [1]:
import os
import getpass
import time
import numpy as np

from openai import OpenAI

os.environ["TOKENIZERS_PARALLELISM"] = "False"

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def ask_openai(question: str) -> str:
    prompt = f"""
    You are a classification bot. Your job is to classify the following query as either faq, general, blocked, or none. Return only the string label or an empty string if no match.

    general is defined as request requiring customer service.
    faq is defined as a request for commonly asked account questions.
    blocked is defined as a request for prohibited information.

    query: "{question}"
    """
    response = client.responses.create(
      model="gpt-4o-mini",
      input=prompt,
    )
    return response

In [10]:
with open("resources/ecom_test_data.json", "r") as f:
    test_data = json.load(f)


res = ask_openai(test_data[0]["query"])
res.output_text

13:23:11 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


'faq'

In [12]:
res.usage.model_dump()

{'input_tokens': 99,
 'input_tokens_details': {'cached_tokens': 0},
 'output_tokens': 2,
 'output_tokens_details': {'reasoning_tokens': 0},
 'total_tokens': 101}

In [13]:
import time

INPUT_TOKEN_PRICE = (0.15 / 1_000_000)
OUTPUT_TOKEN_PRICE = (0.60 / 1_000_000)

def calc_cost_rough(openai_response):
    return openai_response.usage.input_tokens * INPUT_TOKEN_PRICE + openai_response.usage.output_tokens * OUTPUT_TOKEN_PRICE

def test_classifier(classifier, test_data, is_router=False):
    correct = 0
    times = []
    costs = []

    for data in test_data:
        start = time.time()
        if is_router:
            prediction = classifier(data["query"]).name
        else:
            openai_response = ask_openai(data["query"])
            prediction = openai_response.output_text
            costs.append(calc_cost_rough(openai_response))
        
        if not prediction or prediction.lower() == "none":
            prediction = ""

        times.append(time.time() - start)
        print(f"Expected | Observed: {data['query_match']} | {prediction.lower()}")
        if prediction.lower() == data["query_match"]:
            correct += 1

    accuracy = correct / len(test_data)
    avg_time = np.mean(times)
    cost = np.sum(costs) if costs else 0
    return accuracy, avg_time, round(cost, 4)

In [14]:
llm_accuracy, llm_avg_time, llm_cost = test_classifier(ask_openai, test_data)

13:23:43 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | faq
13:23:43 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | faq
13:23:44 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | faq
13:23:44 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | faq
13:23:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | general
13:23:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: faq | faq
13:23:46 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | Observed: general | general
13:23:46 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Expected | 

In [15]:
llm_accuracy, llm_avg_time, llm_cost

(0.8, 0.5609435558319091, 0.0003)

In [16]:
router_accuracy, router_avg_time, router_cost = test_classifier(ecom_router, test_data, is_router=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: faq | faq


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: faq | 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Expected | Observed: blocked | blocked


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: blocked | blocked


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: blocked | blocked


In [17]:
router_accuracy, router_avg_time, router_cost

(0.8, 0.08916561603546143, 0)

In [18]:
speed_improvement = (llm_avg_time / router_avg_time)
speed_improvement

6.291029892159554

In [19]:
from redisvl.extensions.router.schema import DistanceAggregationMethod
from redisvl.extensions.router import RoutingConfig

# toggle aggregation method
ecom_router.update_routing_config(
    RoutingConfig(aggregation_method=DistanceAggregationMethod.min, max_k=3)
)

In [20]:
router_accuracy_min, router_avg_time_min, _  = test_classifier(ecom_router, test_data, is_router=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: faq | faq


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: faq | 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Expected | Observed: blocked | blocked


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: blocked | blocked


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Expected | Observed: blocked | blocked


In [21]:
router_accuracy_min, router_avg_time_min

(0.8, 0.03544937372207642)

## Cleanup

In [22]:
ecom_router.delete()